In [1]:
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv('fake_news.csv')

In [7]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Sentiment analysis prediction using Bi-directional LSTM

In [9]:
# checking for null values
data.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [10]:
# checking the size of the data

data.shape

(20800, 5)

In [13]:
# dropping the null values in the data

data.dropna(inplace=True)

In [15]:
# defining our independent features

x = data.drop('label', axis=1) # indepent features
y = data['label'] # dependent feature

In [17]:
# checking the imbalance in the data (if any)

y.value_counts(1)*100

,proportion
label,
0,56.663932
1,43.336068


In [23]:
# importing all the essential libraries

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [24]:
# defining the vocabulary size

voc_size = 5000

In [25]:
messages=x.copy()

In [29]:
# importing the necessary libraries for stemming

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer() # intializing the PorterStemmer for stemming
corpus = [] # empty list for stemmed output

for index in messages.index: # iterate through the index of the messages
  print(index)
  # Access the row using the actual index
  review = re.sub('[^a-zA-Z]',' ', messages['title'][index]).lower().split() # taking every word without special characters, then lowering it, and then splitting it
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # applying it to stopwords
  review = ' '.join(review)
  corpus.append(review) # appending the data to the corpus

Streaming output truncated to the last 5000 lines.
15115
15116
15117
15118
15119
15120
15123
15124
15125
15126
15127
15128
15129
15130
15131
15132
15133
15134
15136
15137
15138
15139
15140
15141
15142
15143
15144
15145
15146
15147
15149
15150
15151
15153
15154
15155
15156
15157
15158
15159
15161
15162
15163
15164
15165
15166
15167
15168
15169
15170
15171
15172
15174
15175
15176
15177
15178
15179
15181
15182
15183
15184
15185
15186
15187
15188
15189
15191
15192
15193
15194
15195
15196
15197
15199
15200
15201
15202
15203
15204
15205
15206
15207
15208
15209
15211
15212
15213
15214
15215
15216
15217
15218
15219
15220
15221
15222
15223
15224
15225
15226
15227
15228
15229
15230
15231
15232
15233
15234
15235
15236
15237
15239
15240
15241
15242
15243
15244
15245
15246
15247
15248
15249
15250
15251
15252
15253
15254
15255
15256
15257
15258
15259
15260
15262
15265
15266
15267
15268
15269
15270
15271
15272
15273
15274
15275
15276
15277
15278
15279
15280
15281
15282
15284
15285
15286
15287
15288
1

In [34]:
# applying one-hot encoding to the data
one_hot_representation = [one_hot(words, voc_size) for words in corpus]
one_hot_representation

[[3888, 1285, 2657, 2969, 2540, 3165, 2259, 1176, 868, 1737],
 [2920, 1879, 1058, 2618, 573, 2343, 2137],
 [2299, 2156, 2833, 995],
 [4713, 2892, 3119, 1634, 1581, 3467],
 [2413, 573, 2647, 1209, 1295, 386, 573, 1050, 2786, 4750],
 [3497,
  4305,
  2908,
  4086,
  270,
  198,
  3299,
  2620,
  1041,
  292,
  4216,
  1749,
  1676,
  2232,
  2137],
 [49, 655, 926, 1109, 1296, 2549, 3081, 998, 605, 4764, 3736],
 [1432, 4809, 4734, 2585, 3809, 558, 198, 23, 605, 4764, 3736],
 [1229, 4783, 2256, 670, 2191, 2000, 4827, 342, 198, 2549],
 [2528, 1101, 576, 4105, 3678, 779, 2532, 2848],
 [3621, 659, 443, 1872, 3010, 1524, 1777, 243, 674, 1772, 502],
 [1634, 4161, 2540, 2000, 198, 3809],
 [3784, 465, 4157, 2160, 1369, 380, 4934, 4216, 1142],
 [2756, 3862, 3563, 1588, 832, 1312, 2397, 605, 4764, 3736],
 [3171, 2962, 2605, 695, 3267, 605, 4764, 3736],
 [2405, 3490, 2272, 236, 3402, 88, 4827, 1807, 948, 3351],
 [1412, 4876, 1879],
 [523, 3245, 909, 3097, 198, 1636, 6, 2137],
 [1490, 3900, 1058, 175

In [35]:
# reminder: we always have to one-hot encode the data before passing it through an embedding layer

In [36]:
#applying the embedding representation
sent_length = 20  #defining the max sentence length (user-defined)

#applying padding to ensure each sentence is of the same length
embedded_docs = pad_sequences(one_hot_representation, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1176,  868, 1737],
       [   0,    0,    0, ...,  573, 2343, 2137],
       [   0,    0,    0, ..., 2156, 2833,  995],
       ...,
       [   0,    0,    0, ...,  605, 4764, 3736],
       [   0,    0,    0, ..., 4698, 1498, 3236],
       [   0,    0,    0, ..., 2894, 4673, 1566]], dtype=int32)

# Model Building

In [38]:
# defining the model parameters and layers
embedding_vector_features = 40    #defining the vector size, to convert the sentence indexes to vectors
model = Sequential()    #sequential layer
model.add(Embedding(voc_size, embedding_vector_features))    #embedding layer
model.add(Bidirectional(LSTM(100)))    #Bidirectional LSTM layer
model.add(Dense(1,activation='sigmoid'))   #Dense layer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [41]:
# splitting the embedded docs variable
# converting into an numpy array
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [42]:
# train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_final,y_final, test_size=0.3, random_state=1)

In [43]:
# fitting the model on our training data
model.fit(x_train,y_train, epochs=20, batch_size=32)

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.8172 - loss: 0.3462
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 38ms/step - accuracy: 0.9518 - loss: 0.1275
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 40ms/step - accuracy: 0.9670 - loss: 0.0940
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 35ms/step - accuracy: 0.9772 - loss: 0.0627
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - accuracy: 0.9877 - loss: 0.0436
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.9928 - loss: 0.0254
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.9955 - loss: 0.0158
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 46ms/step - accuracy: 0.9966 - loss: 0.0119
Epoch 9/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - accuracy: 0.9976 - loss: 0.0083
Epoch 10/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.9980 - loss: 0.0064
Epoch 11/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.9977 - loss: 0.0077
Epoch 12/20
400/400 ━━━━━━━━━━

In [45]:
# predicting on the test data
pred_test = np.where(model.predict(x_test)>0.5,1,0)

172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [46]:
# checking model performance on test data
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test, pred_test)
custom_cm = pd.DataFrame([[cm[1,1],cm[1,0]],
                          [cm[0,1],cm[0,0]]],
                         columns=['Predicted Positive','Predicted Negative'],
                         index=['Actual Positive','Actual Negative'])
custom_cm

,Predicted Positive,Predicted Negative
Actual Positive,2146,214
Actual Negative,257,2869


In [47]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3126
           1       0.89      0.91      0.90      2360

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486

